In [1]:
import pandas as pd
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.graph_objects as po

import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')
df.head()

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California


In [3]:
app = dash.Dash(__name__)
app.title = "Automobile Statistics Dashboard"

In [4]:
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]

In [5]:
year_list = [i for i in range(1980, 2024, 1)]

In [6]:
app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign':'center', 'color': '#503D36', 'font-size':'24'}),
    html.Div([
        html.Label("Select Statistics: "),
        dcc.Dropdown(
            id='dropdown-statistics',
            options = dropdown_options,
            value = 'Select Statistics',
            placeholder='Select a report type'
        )
    ]),
     html.Div([
        html.Label("Select Year: "),
        dcc.Dropdown(
            id='select-year',
            options = [{'label': i, 'value': i} for i in year_list],
            value = 'select-year',
            placeholder='select-year'
        )
    ]),
         html.Div([
             html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
         ])

])

In [7]:
@app.callback(Output(component_id='select-year', component_property='disabled'),
             Input(component_id='dropdown-statistics', component_property='value'))
def update_input_container(selected_statistics):
    if selected_statistics == "Yearly Statistics":
        return False
    else:
        return True

In [8]:
@app.callback(Output(component_id='output-container', component_property='children'),
             [Input(component_id='dropdown-statistics', component_property='value'),
              Input(component_id='select-year', component_property='value')])
def update_output_container(dropdown_statistics, select_year):
    df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')
    if dropdown_statistics == 'Recession Period Statistics':
        print("In Recession Period Statistics", select_year)
        recession_data = df[df['Recession'] == 1]
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
             x='Year',
             y='Automobile_Sales',
             title="Average Automobile Sales fluctuation over Recession Period"))
        
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()                
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales,
            x='Vehicle_Type',
            y='Automobile_Sales',
            title="Average number of vehicles sold by vehicle type "))
        
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                    values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="Total expenditure share by vehicle type during recessions"
                )
        )
        
        unemp_data= recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data,
        x='Vehicle_Type',
        y='Automobile_Sales',
        color='unemployment_rate',
        labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
        title='Effect of Unemployment Rate on Vehicle Type and Sales'))
        
        return [
            html.Div([
                html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)],style={'display': 'flex'})
            ])]
    elif select_year and dropdown_statistics == 'Yearly Statistics':
        print("In select_year", select_year)
        yearly_data = df[df['Year'] == select_year]
        yas= df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x="Year", y="Automobile_Sales", title="Yearly Automobile sales using line chart for the whole period."))

        mas=df.groupby(["Year", "Month"])['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas,
            x="Month",
            y='Automobile_Sales',
            title='Total Monthly Automobile Sales'))

         # Hint:Use the columns Year and Automobile_Sales
        avr_vdata=yearly_data.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
        Y_chart3 = dcc.Graph( figure=px.bar(avr_vdata, x="Vehicle_Type", y="Automobile_Sales", color="Vehicle_Type" ,title='Average Vehicles Sold by Vehicle Type in the year {}'.format(select_year)))

        exp_data=yearly_data.groupby('Vehicle_Type')["Advertising_Expenditure"].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, 
            values="Advertising_Expenditure",
            names='Vehicle_Type',
            title='Total Advertisment Expenditure for Each Vehicle'))
        return [
             html.Div([
                html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display':'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
             ])
            ]
    else:
        print("In else")
        return None

        

In [9]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8090)